In [24]:
import re
import json
import random
import string

from vllm import LLM
from vllm.sampling_params import SamplingParams

### Load model

In [13]:
model_name = "Qwen/Qwen3-30B-A3B"

sampling_params = SamplingParams(
    max_tokens=8192,
    temperature=0.0,
)

llm = LLM(
    model=model_name,
)

config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

INFO 09-05 14:49:04 [config.py:853] This model supports multiple tasks: {'reward', 'classify', 'embed', 'generate', 'score'}. Defaulting to 'generate'.


tokenizer_config.json: 0.00B [00:00, ?B/s]

INFO 09-05 14:49:05 [config.py:1467] Using max model len 40960
INFO 09-05 14:49:12 [config.py:2267] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 09-05 14:49:12 [config.py:4566] full_cuda_graph is not supported with cascade attention. Disabling cascade attention.


vocab.json: 0.00B [00:00, ?B/s]

...

WARNING 09-05 14:49:14 [utils.py:2613] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 09-05 14:49:17 [__init__.py:244] Automatically detected platform rocm.
INFO 09-05 14:49:26 [core.py:459] Waiting for init message from front-end.
INFO 09-05 14:49:26 [core.py:69] Initializing a V1 LLM engine (v0.9.2.dev364+gb432b7a28) with config: model='Qwen/Qwen3-30B-A3B', speculative_config=None, tokenizer='Qwen/Qwen3-30B-A3B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_c

Loading safetensors checkpoint shards:   0% Completed | 0/16 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/16 [00:01<00:24,  1.62s/it]
Loading safetensors checkpoint shards:  12% Completed | 2/16 [00:03<00:24,  1.74s/it]
Loading safetensors checkpoint shards:  19% Completed | 3/16 [00:05<00:22,  1.77s/it]
Loading safetensors checkpoint shards:  25% Completed | 4/16 [00:07<00:21,  1.79s/it]
Loading safetensors checkpoint shards:  31% Completed | 5/16 [00:07<00:15,  1.37s/it]
Loading safetensors checkpoint shards:  38% Completed | 6/16 [00:09<00:14,  1.46s/it]
Loading safetensors checkpoint shards:  44% Completed | 7/16 [00:11<00:14,  1.56s/it]
Loading safetensors checkpoint shards:  50% Completed | 8/16 [00:12<00:13,  1.64s/it]
Loading safetensors checkpoint shards:  56% Completed | 9/16 [00:14<00:11,  1.68s/it]
Loading safetensors checkpoint shards:  62% Completed | 10/16 [00:16<00:10,  1.71s/it]
Loading safetensors checkpoint shards:  69% Completed | 11/16

INFO 09-05 14:50:43 [default_loader.py:272] Loading weights took 27.31 seconds
INFO 09-05 14:50:43 [gpu_model_runner.py:1782] Model loading took 77.5273 GiB and 75.943223 seconds
INFO 09-05 14:50:51 [backends.py:509] Using cache directory: /root/.cache/vllm/torch_compile_cache/fe72b23fcf/rank_0_0/backbone for vLLM's torch.compile
INFO 09-05 14:50:51 [backends.py:520] Dynamo bytecode transform time: 7.42 s
INFO 09-05 14:51:23 [backends.py:181] Cache the graph of shape None for later use
INFO 09-05 14:51:23 [backends.py:193] Compiling a graph for general shape takes 28.65 s
INFO 09-05 14:51:30 [monitor.py:34] torch.compile takes 36.08 s in total
WARNING 09-05 14:51:43 [fused_moe.py:683] Using default MoE config. Performance might be sub-optimal! Config file not found at /usr/local/lib/python3.12/dist-packages/vllm/model_executor/layers/fused_moe/configs/E=128,N=768,device_name=AMD_Instinct_MI300X.json
INFO 09-05 14:51:45 [gpu_worker.py:232] Available KV cache memory: 88.58 GiB
INFO 09-05

Capturing CUDA graphs: 100%|██████████| 67/67 [00:24<00:00,  2.75it/s]


INFO 09-05 14:52:09 [gpu_model_runner.py:2306] Graph capturing finished in 24 secs, took 0.40 GiB
INFO 09-05 14:52:10 [core.py:172] init engine (profile, create kv cache, warmup model) took 86.46 seconds


### Tools

In [14]:
def generate_random_id(length=9):
    characters = string.ascii_letters + string.digits
    random_id = "".join(random.choice(characters) for _ in range(length))
    return random_id


# simulate an API that can be called
def get_current_weather(city: str, state: str, unit: "str"):
    return (
        f"The weather in {city}, {state} is 85 degrees {unit}. It is "
        "partly cloudly, with highs in the 90's."
    )

In [15]:
tool_functions = {"get_current_weather": get_current_weather}

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to find the weather for, e.g. 'San Francisco'",
                    },
                    "state": {
                        "type": "string",
                        "description": "the two-letter abbreviation for the state that the city is"
                        " in, e.g. 'CA' which would mean 'California'",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The unit to fetch the temperature in",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["city", "state", "unit"],
            },
        },
    }
]

### Tool calls

In [49]:
messages = [
    {
        "role": "user",
        "content": "Can you tell me what the temperate will be in Dallas, in fahrenheit?",
    }
]

In [ ]:
outputs = llm.chat(
    messages,
    sampling_params=sampling_params,
    tools=tools,
    chat_template_kwargs={"enable_thinking": False},
)

output = outputs[0].outputs[0].text.strip()

match = re.search(r'<tool_call>(.*?)</tool_call>', output, re.DOTALL)
if match:
    tool_output = match.group(1).strip()

messages.append(
    {
        "role": "assistant",
        "content": tool_output,
    }
)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0% 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [51]:
tool_output

'{"name": "get_current_weather", "arguments": {"city": "Dallas", "state": "TX", "unit": "fahrenheit"}}'

In [56]:
tool_calls = [json.loads(tool_output)]
tool_answers = [
    tool_functions[call["name"]](**call["arguments"]) for call in tool_calls
]

In [59]:
messages.append(
    {
        "role": "tool",
        "content": "\n\n".join(tool_answers),
        "tool_call_id": generate_random_id(),
    }
)

outputs = llm.chat(messages, sampling_params, tools=tools, chat_template_kwargs={"enable_thinking": False})

print(outputs[0].outputs[0].text.strip())

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0% 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

The current temperature in Dallas, TX is 85 degrees Fahrenheit. The weather is partly cloudy, with high temperatures expected in the 90s.
